In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, time

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import make_scorer

import matplotlib.pyplot as plt


In [ ]:
data_root = "/kaggle/input/lish-moa/"
train_X = pd.read_csv(data_root + 'train_features.csv')
train_Y = pd.read_csv(data_root + 'train_targets_scored.csv')
test_X = pd.read_csv(data_root + 'test_features.csv')
sample_submission = pd.read_csv(data_root + 'sample_submission.csv')

In [ ]:
# change cp_time columns to string
train_X['cp_time'] = train_X['cp_time'].apply(lambda x:str(x))
test_X['cp_time'] = test_X['cp_time'].apply(lambda x:str(x))

# pd.get_dummies gets one-hot encoding for categorical columns
# we also the remove the original categorical columns
train_X = train_X.join(pd.get_dummies(train_X[['cp_time','cp_type','cp_dose']])).drop(['cp_time','cp_type','cp_dose'],axis=1)
test_X = test_X.join(pd.get_dummies(test_X[['cp_time','cp_type','cp_dose']])).drop(['cp_time','cp_type','cp_dose'],axis=1)

# get the ids from the test/training data and then remove those columns
test_ids = test_X['sig_id']
test_X.drop(['sig_id'],axis=1,inplace=True)

train_ids = train_X['sig_id']
train_X.drop(['sig_id'],axis=1,inplace=True)
train_Y.drop(['sig_id'],axis=1,inplace=True)

print(train_X, test_X)
print(train_X.columns, test_X.columns)
print(train_X.shape, train_Y.shape, test_X.shape)


In [ ]:
# we only scale the real-valued features and not the categorical ones
g_cols = [col for col in train_X.columns if col.startswith('g-')]
c_cols = [col for col in train_X.columns if col.startswith('c-')]
transform_feature_list = g_cols + c_cols


def scale_and_PCA(pca_num_components, train, test, cols_to_transform, transformed_col_name):
    # create data by stacking rows from both train and test, for the required columns
    data = pd.concat([train[cols_to_transform], test[cols_to_transform]], axis=0).reset_index(drop=True)
    n = train.shape[0]
    
    # scale
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)

    # PCA
    pca = PCA(pca_num_components)
    pca_data = pca.fit_transform(scaled_data)

    train_trans = pca_data[:n, :]
    test_trans = pca_data[n:, :]

    return train_trans, test_trans

# first scale and PCA on "g-" features
# we use top 10 "g-" features after PCA
train_X_g, test_X_g = scale_and_PCA(10, train_X, test_X, g_cols, 'g_pca')

# next, scale and tranform the "c-" features
# we use top 5 "c-" features after PCA
train_X_c, test_X_c = scale_and_PCA(5, train_X, test_X, c_cols, 'c_pca')

# concatenate the "g-" and "c-" columns
data_train = np.concatenate((train_X_g, train_X_c), axis=1)
data_test = np.concatenate((test_X_g, test_X_c), axis=1)

# now concatenate train and test rows
n = data_train.shape[0]
data = np.concatenate((data_train, data_test), axis=0)

# categorical columns are the last 7 columns of train/test data (after the preprocessing that we did earlier)
cat_data = np.concatenate((train_X.iloc[:, -7:].to_numpy(), test_X.iloc[:, -7:].to_numpy()), axis=0)

# add back the 7 categorical columns in front of the real-valued columns
transformed_data = np.concatenate((cat_data, data), axis=1)

# seaparate the train/test data
transformed_train_data = transformed_data[:n, :]
transformed_test_data = transformed_data[n:, :]

print(transformed_train_data.shape, transformed_test_data.shape)

# **Define the average column-wise log-loss function**

In [ ]:
def average_log_loss (y_true, y_pred):
    print(y_true.shape, y_pred.shape)
    num_samples, num_outputs = y_true.shape
    loss = 0.00
    for i in range(num_outputs):
        loss += log_loss(y_true[:, i], y_pred[:, i])
    loss /= num_outputs
    return loss


scorer = make_scorer(average_log_loss, greater_is_better=False)


In [ ]:
transformed_train_targets = train_Y.values

In [ ]:
# create a list of class weights
class_weights = []
for i in range(transformed_train_targets.shape[1]):
    class_weights.append({0: 0.4, 1:0.6})

clf = RandomForestClassifier(n_estimators=250, max_depth=10, max_features='log2', class_weight=class_weights)

start = time.time()
clf.fit(transformed_train_data, transformed_train_targets)
stop = time.time()
print("Time taken to fit StackingClassifier: ", stop-start, " seconds")

probs = clf.predict(transformed_test_data)

# the first index is the probability of output 1 for a feature
print(probs, probs.shape)
preds = np.array(probs) #[:, :, 1]
sample_submission[sample_submission.columns.to_list()[1:]] = preds
sample_submission.to_csv('submission.csv',index=False)